![walmartecomm](walmartecomm.jpg)

Here is the short backgroun about the project: Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data.

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [12]:
import pandas as pd
import os 

grocery_sales=pd.read_csv('grocery_sales.csv')

# Extract function to read the parquet data and then merge with the grocery sales data 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Calling the extract() function and storing it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

In [116]:
# Checking the output of the merged file
print(merged_df.head())

   index  Store_ID       Date  Dept  ...         CPI  Unemployment  Type      Size
0      0         1 2010-02-05     1  ...  211.096358         8.106   3.0  151315.0
1      1         1 2010-02-05    26  ...  211.096358         8.106   3.0  151315.0
2      2         1 2010-02-05    17  ...  211.096358         8.106   3.0  151315.0
3      3         1 2010-02-05    45  ...  211.096358           NaN   3.0  151315.0
4      4         1 2010-02-05    28  ...  211.096358           NaN   3.0  151315.0

[5 rows x 17 columns]


In [117]:
# Checking which columns have missing values
merged_df.isna().sum()

index            0
Store_ID         0
Date            39
Dept             0
Weekly_Sales    38
IsHoliday        0
Temperature      0
Fuel_Price       0
MarkDown1        0
MarkDown2        0
MarkDown3        0
MarkDown4        1
MarkDown5        1
CPI             47
Unemployment    37
Type             1
Size             1
dtype: int64

In [118]:
# Creating the transform() function with one parameter: "raw_data"
# This function takes the merged file as an input, fills missing numerical values, adds a column "Month",  then filters the rows with weekly sales over $10000 and drops the unncessary columns
def transform(raw_data):
  merged_df['Date']=merged_df['Date'].fillna(method='ffill')
  merged_df['Temperature']=merged_df['Temperature'].fillna(method='ffill')
  merged_df['CPI']=merged_df['CPI'].fillna(method='ffill')
  merged_df['Unemployment']=merged_df['Unemployment'].fillna(method='ffill')
  merged_df['Unemployment']=merged_df['Unemployment'].fillna(method='ffill')
  merged_df['MarkDown4']=merged_df['MarkDown4'].fillna(method='ffill')
  merged_df['MarkDown5']=merged_df['MarkDown5'].fillna(method='ffill')  
  merged_df['Size']=merged_df['Size'].fillna(method='ffill')
  merged_df['Type']=merged_df['Type'].fillna(method='ffill')
  merged_df['Weekly_Sales']=merged_df['Weekly_Sales'].fillna(merged_df.groupby(['Store_ID', 'Dept','Date'])['Weekly_Sales'].transform('mean'))  
  merged_df['Month']=merged_df['Date'].dt.month
  merged_df2=merged_df[merged_df['Weekly_Sales']>10000]
  merged_df3=merged_df2[['Store_ID', 'Month', 'Dept', 'IsHoliday', 'Weekly_Sales', 'CPI', 'Unemployment']]
  return merged_df3
  
  
pass

In [119]:
# Calling  the transform() function and passing the merged DataFrame
clean_data = transform(merged_df)

In [120]:
# Checking the clean data
print(clean_data.head())

   Store_ID  Month  Dept  IsHoliday  Weekly_Sales         CPI  Unemployment
0         1      2     1          0      24924.50  211.096358         8.106
1         1      2    26          0      11737.12  211.096358         8.106
2         1      2    17          0      13223.76  211.096358         8.106
5         1      2    79          0      46729.77  211.096358         8.106
6         1      2    55          0      21249.31  211.096358         8.106


In [121]:
# Creating the avg_weekly_sales_per_month function that takes in the cleaned data from the last step
def avg_weekly_sales_per_month(clean_data):
    cleaned_data1=clean_data.groupby('Month')
    result=cleaned_data1.agg(Avg_Sales=('Weekly_Sales','mean'))
    result1=result.reset_index()
    rounded=round(result1, 2)
    agg_data=rounded
    return agg_data
    pass

In [122]:
# Calling the avg_weekly_sales_per_month() function and passing the cleaned DataFrame
agg_data=avg_weekly_sales_per_month(clean_data)

In [123]:
# CHecking the reulst of the avg_weekly_sales_per_month() function
print(agg_data)

    Month  Avg_Sales
0       1   33174.18
1       2   34340.44
2       3   33227.31
3       4   33413.25
4       5   33339.89
5       6   34582.47
6       7   33930.77
7       8   33644.79
8       9   33266.59
9      10   32731.06
10     11   36594.03
11     12   39248.98


In [124]:
# Creating the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    saved_full_data=full_data.to_csv(full_data_file_path, index=False)
    saved_agg_data=agg_data.to_csv(agg_data_file_path, index=False)
    
    pass

In [125]:
# Calling the load() function and passing the cleaned and aggregated DataFrames with their paths
load(clean_data, 'clean_data.csv',agg_data, 'agg_data.csv')

In [126]:
# Now, we need to check whether the two csv files from the load() function exist in the current working directory 
# Creating the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    file_exists=os.path.exists(file_path)
    print(file_exists)
    pass

In [127]:
# Calling the validation() function and passing first, the cleaned DataFrame path, and then the aggregated DataFrame path
validation('clean_data.csv')
validation('agg_data.csv')

True
True
